In [38]:
import concurrent.futures
import urllib
import json
import time
import os
import os.path
import re
from bs4 import BeautifulSoup

In [39]:
BASE_DIR = '/data/neon/customer_images/pinterest'
QUERIES = ['nursery', 'cake', 'woodworking', 'sailboat', 'travel', 'hiking', 'spring dress', 'spring', 
           'italian recipes', 'dogs']
API_TOKEN = 'AaBgobGn8YvYxWKnOX6ruHNdaW3_FHMOb9gvjGRDT8aQJsAr2QAAAAA'
BOARDS_PER_QUERY = 10
IMAGES_PER_BOARD = 200

In [40]:
def search_for_boards(query, max_boards=None):
    '''Search for boards. Returns a list of board names'''
    url = 'https://www.pinterest.com/search/boards/?%s' % urllib.urlencode({'q': query})
    page = urllib.urlopen(url).read()
    soup = BeautifulSoup(page, 'html.parser')
    for script_tag in soup.find_all('script'):
        if script_tag.attrs.get('type') == 'application/json' and script_tag.attrs.get('id') == 'jsInit1':
            data = json.loads(script_tag.string)
            rv = [x['url'].strip('/') for x in data['tree']['data']['results']]
            if max_boards:
                rv = rv[:max_boards]
            return rv
        
    raise IOError('Could not find boards for query %s' % query)


In [41]:
def iterate_board_image_urls(board, max_images=None):
    url = 'https://api.pinterest.com/v1/boards/%s/pins/?cursor=&access_token=%s&fields=image' % (board, API_TOKEN)
    
    image_count = 0
    while url:
        data = json.loads(urllib.urlopen(url).read())
        url = data['page'].get('next')
        for entry in data['data']:
            if max_images and image_count >= max_images:
                return
            try:
                yield entry['image']['original']['url']
                image_count += 1
            except KeyError:
                pass


In [42]:
def process_board(query, board):
    d = os.path.join(BASE_DIR, query, board)
    if not os.path.exists(d):
        os.makedirs(d)
        
    for url in iterate_board_image_urls(board, max_images=IMAGES_PER_BOARD):
        fn = os.path.join(d, os.path.basename(url))
        if not os.path.exists(fn):
            urllib.urlretrieve(url, fn)

In [43]:
def process_query(query):
    for board in search_for_boards(query, BOARDS_PER_QUERY):
        print 'Processing board %s' % board
        process_board(query, board)


In [45]:
with concurrent.futures.ThreadPoolExecutor(6) as executor:
    futs = [executor.submit(process_query, x) for x in QUERIES]
    concurrent.futures.wait(futs)

Processing board instructables/woodworking
Processing board plumprettysugar/nursery
Processing board damndelicious/cake
Processing board dvf/travel
Processing board halinkis/sailboat
Processing board anamwagner/hiking
Processing board spearmintbaby/nursery
Processing board aptapothecary/nursery
Processing board CoyoteAsh/sailboat
Processing board designlovelife/nursery
Processing board lana159/sailboat
Processing board myitchytravelft/hiking
Processing board family_handyman/woodworking
Processing board shoppinkblush/nursery
Processing board nasti/travel
Processing board jacquelynclark/cake
Processing board kellyjoartist/sailboat
Processing board plumprettysugar/cake
Processing board lmmckenna60/sailboat
Processing board diyboards/woodworking
Processing board survivallife/hiking
Processing board juneandjanuary_/nursery
Processing board maribelal/hiking
Processing board galmeetsglam/travel
Processing board twopeasandpod/cake
Processing board stazerbeam/liveaboard-sailboat
Processing boar